# NetworkX

>[NetworkX](https://networkx.org/) is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

This notebook goes over how to do question answering over a graph data structure.

## Setting up

We have to install a Python package.

In [ ]:
%pip install --upgrade --quiet  networkx

## Create the graph

In this section, we construct an example graph. At the moment, this works best for small pieces of text.

In [1]:
from langchain.indexes import GraphIndexCreator
from langchain_openai import OpenAI

In [2]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [3]:
with open("../../../modules/state_of_the_union.txt") as f:
    all_text = f.read()

We will use just a small snippet, because extracting the knowledge triplets is a bit intensive at the moment.

In [4]:
text = "\n".join(all_text.split("\n\n")[105:108])

In [5]:
text

'It won’t look like much, but if you stop and look closely, you’ll see a “Field of dreams,” the ground on which America’s future will be built. \nThis is where Intel, the American company that helped build Silicon Valley, is going to build its $20 billion semiconductor “mega site”. \nUp to eight state-of-the-art factories in one place. 10,000 new good-paying jobs. '

In [6]:
graph = index_creator.from_text(text)

We can inspect the created graph.

In [7]:
graph.get_triples()

[('Intel', '$20 billion semiconductor "mega site"', 'is going to build'),
 ('Intel', 'state-of-the-art factories', 'is building'),
 ('Intel', '10,000 new good-paying jobs', 'is creating'),
 ('Intel', 'Silicon Valley', 'is helping build'),
 ('Field of dreams',
  "America's future will be built",
  'is the ground on which')]

## Querying the graph
We can now use the graph QA chain to ask question of the graph

In [8]:
from langchain.chains import GraphQAChain

In [9]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [10]:
chain.run("what is Intel going to build?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Intel
Full Context:
Intel is going to build $20 billion semiconductor "mega site"
Intel is building state-of-the-art factories
Intel is creating 10,000 new good-paying jobs
Intel is helping build Silicon Valley

> Finished chain.


' Intel is going to build a $20 billion semiconductor "mega site" with state-of-the-art factories, creating 10,000 new good-paying jobs and helping to build Silicon Valley.'

## Save the graph
We can also save and load the graph.

In [7]:
graph.write_to_gml("graph.gml")

In [8]:
from langchain.indexes.graph import NetworkxEntityGraph

In [9]:
loaded_graph = NetworkxEntityGraph.from_gml("graph.gml")

In [10]:
loaded_graph.get_triples()

[('Intel', '$20 billion semiconductor "mega site"', 'is going to build'),
 ('Intel', 'state-of-the-art factories', 'is building'),
 ('Intel', '10,000 new good-paying jobs', 'is creating'),
 ('Intel', 'Silicon Valley', 'is helping build'),
 ('Field of dreams',
  "America's future will be built",
  'is the ground on which')]

In [ ]:
loaded_graph.get_number_of_nodes()

In [ ]:
loaded_graph.add_node("NewNode")

In [ ]:
loaded_graph.has_node("NewNode")

In [ ]:
loaded_graph.remove_node("NewNode")

In [ ]:
loaded_graph.get_neighbors("Intel")

In [ ]:
loaded_graph.has_edge("Intel", "Silicon Valley")

In [ ]:
loaded_graph.remove_edge("Intel", "Silicon Valley")

In [ ]:
loaded_graph.clear_edges()

In [ ]:
loaded_graph.clear()